In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install contractions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 7.4 MB/s eta 0:00:00


In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import contractions
from tensorflow.keras.preprocessing.text import Tokenizer
import re
import numpy as np
from sklearn.metrics import f1_score,classification_report, confusion_matrix,precision_score, recall_score
from tensorflow import keras

In [ ]:
#df = pd.read_csv('/content/drive/MyDrive/post classification/processed_data/post-ontology-label.csv')
#df

df = pd.read_csv('/content/drive/MyDrive/post classification/processed_data/posts-ontology-manual-label.csv')
df



,post,prefLabel,compulsion,obsession,expand1
0,Hi -\nSo I haven't been on here since December...,"['depression', 'weight gain', 'Medication', 'a...",0,1,"Aggressive obsession, Contamination obsession,..."
1,"Hi all, hope you're all having a wonderful ban...","['compulsion', 'anger', 'symptom', 'compassion...",1,1,"Aggressive intrusive thoughts, Contamination i..."
2,"Hi, \nFirst, I hope everyone managed to have s...","['hope', 'happiness', 'guilt', 'fear', 'obsess...",0,1,"Aggressive obsession, Contamination obsession,..."
3,Hello everyone. I could really use your help r...,"['Treatment', 'hope', 'Thought', 'obsession', ...",0,1,"Aggressive obsession, Contamination obsession,..."
4,"Though it comes in many flavors, one of the mo...","['quality', 'Intrusive thoughts', 'fall', 'beh...",1,1,"Aggressive obsession, Contamination obsession,..."
...,...,...,...,...,...
9991,https://itunes.apple.com/gb/podcast/dhamma-tal...,"['interest', 'Thought', 'OCD', 'cognitive proc...",0,0,"morbid thoughts, Mental image, Homicidal thoug..."
9992,Hi all\nThere are lots of helpful topics on th...,['OCD'],0,0,NaN
9993,"aww guys i am sorry , i feel that i am always ...","['OCD', 'sadness', 'weakness', 'anxiety', 'anx...",0,0,"castration anxiety, performance anxiety, koro,..."
9994,My name is Ian I have suffered with OCD for mo...,"['OCD', 'hope', 'fear']",1,0,"terror, panic, panic, moral panic"


In [ ]:
import re
#define the process of text cleaning
def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r' ',text)

#Clean Text
def clean_text(data):
    # convert catacter to lowercase
    data['clean_text']=data['yy'].str.lower()
    #remove URLS
    data['clean_text'] = data['clean_text'].apply(lambda elem:re.sub(r"http\S+", "", elem))
    #remove ponctuation
    data['clean_text'] = data['clean_text'].apply(lambda elem:re.sub(r"[^\w\s]", "", elem))
    #remove
    data['clean_text'] = data['clean_text'].apply(lambda elem:re.sub(r'/n',"",elem))
    #remove degits
    data['clean_text'] = data['clean_text'].apply(lambda elem:re.sub(r'\d+',"",elem))
    #remove emojis
    data['clean_text'] = data['clean_text'].apply(lambda elem:deEmojify(elem))
    #remove multiple spaces
    data['clean_text'] = data['clean_text'].apply(lambda elem:re.sub(r'\s+'," ",elem))
    #remove single caracter
    data['clean_text'] = data['clean_text'].apply(lambda elem:re.sub(r'\s+[a-zA-Z]\s+'," ",elem))
    return data

def process_text(text):
    stop_words = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()

    # Handle contractions using the contractions library
    expanded_text = contractions.fix(text)

    # Lowercasing
    expanded_text = expanded_text.lower()

    # Tokenization
    tokens = word_tokenize(expanded_text)

    # Removing Punctuation
    tokens = [word for word in tokens if word.isalnum()]

    # Removing Stop Words
    tokens = [word for word in tokens if word not in stop_words]

    # Lemmatization
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    return ' '.join(tokens)

In [ ]:
df['post'] = df['post'].fillna('')
df['post'] = df['post'].apply(process_text)
df.head()

,post,prefLabel,compulsion,obsession,expand1
0,hi since december since recovering prozac sinc...,"['depression', 'weight gain', 'Medication', 'a...",0,1,"Aggressive obsession, Contamination obsession,..."
1,hi hope wonderful bank holiday whatever howeve...,"['compulsion', 'anger', 'symptom', 'compassion...",1,1,"Aggressive intrusive thoughts, Contamination i..."
2,hi first hope everyone managed joy happiness c...,"['hope', 'happiness', 'guilt', 'fear', 'obsess...",0,1,"Aggressive obsession, Contamination obsession,..."
3,hello everyone could really use help right ide...,"['Treatment', 'hope', 'Thought', 'obsession', ...",0,1,"Aggressive obsession, Contamination obsession,..."
4,though come many flavor one common ocd theme s...,"['quality', 'Intrusive thoughts', 'fall', 'beh...",1,1,"Aggressive obsession, Contamination obsession,..."


In [ ]:
X, Y = df['post'], df[['obsession']]
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((7996,), (7996, 1), (2000,), (2000, 1))

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
num_words = 2000
vect=Tokenizer(num_words=num_words)
vect.fit_on_texts(X_train)
vocab_size = len(vect.word_index) + 1
print(vocab_size)

22629


In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
encoded_docs_train = vect.texts_to_sequences(X_train)
MAX_LEN = 150
padded_docs_train = pad_sequences(encoded_docs_train, maxlen=MAX_LEN, padding='pre')
padded_docs_train.shape

(7996, 150)

In [ ]:
padded_docs_train

array([[   0,    0,    0, ...,  992, 1590,  992],
       [   0,    0,    0, ...,   37,  247,  247],
       [   0,    0,    0, ...,  356,  192,  283],
       ...,
       [   0,    0,    0, ...,   99, 1452,  909],
       [   0,    0,    0, ..., 1002,  555, 1091],
       [   0,    0,    0, ...,   30,  433,  136]], dtype=int32)

In [ ]:
encoded_docs_test =  vect.texts_to_sequences(X_test)
padded_docs_test = pad_sequences(encoded_docs_test, maxlen=MAX_LEN, padding='pre')

In [ ]:
# Index of the document you want to print (change this to the desired document)
document_index = 0

# Retrieve the original text document
original_text = X_train[document_index]

# Retrieve the numerical representation of the document
numerical_representation = padded_docs_train[document_index]

# Print the document and its numerical representation
print("Original Text Document:")
print(original_text)

print("\nNumerical Representation:")
print(numerical_representation)

In [ ]:
from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout

def get_model():
    model = Sequential()
    # Configuring the parameters
    model.add(Embedding(num_words, output_dim=16, input_length=MAX_LEN))
    model.add(LSTM(16, return_sequences=True))
    # Adding a dropout layer
    model.add(Dropout(0.5))
    model.add(LSTM(8))
    # model.add(Dropout(0.5))
    model.add(Dense(4))
    # model.add(Dropout(0.5))
    # Adding a dense output layer with sigmoid activation
    model.add(Dense(1, activation='sigmoid'))
    print(model.summary())
    return model

In [ ]:
obsession_model = get_model()
obsession_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
history = obsession_model.fit(padded_docs_train, y_train, epochs=1000, batch_size=16,validation_split=0.2,
                    callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss',patience=20, min_delta=1e-7),
                              keras.callbacks.ReduceLROnPlateau(factor=0.2, patience=10),
                              keras.callbacks.ModelCheckpoint(filepath='model/lstm_obsession_model.h5',
                                      monitor='val_loss',
                                      save_best_only=True)])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 150, 16)           32000     
                                                                 
 lstm (LSTM)                 (None, 150, 16)           2112      
                                                                 
 dropout (Dropout)           (None, 150, 16)           0         
                                                                 
 lstm_1 (LSTM)               (None, 8)                 800       
                                                                 
 dense (Dense)               (None, 4)                 36        
                                                                 
 dense_1 (Dense)             (None, 1)                 5         
                                                                 
Total params: 34953 (136.54 KB)
Trainable params: 34953 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


400/400 [==============================] - 46s 114ms/step - loss: 0.2202 - acc: 0.9343 - val_loss: 0.2274 - val_acc: 0.9344 - lr: 0.0010
Epoch 3/1000
400/400 [==============================] - 50s 124ms/step - loss: 0.2015 - acc: 0.9443 - val_loss: 0.2368 - val_acc: 0.9325 - lr: 0.0010
Epoch 4/1000
400/400 [==============================] - 49s 124ms/step - loss: 0.2012 - acc: 0.9429 - val_loss: 0.2379 - val_acc: 0.9394 - lr: 0.0010
Epoch 5/1000
400/400 [==============================] - 49s 123ms/step - loss: 0.1901 - acc: 0.9489 - val_loss: 0.2404 - val_acc: 0.9331 - lr: 0.0010
Epoch 6/1000
400/400 [==============================] - 47s 118ms/step - loss: 0.1829 - acc: 0.9495 - val_loss: 0.2259 - val_acc: 0.9400 - lr: 0.0010
Epoch 7/1000
400/400 [==============================] - 48s 120ms/step - loss: 0.1778 - acc: 0.9515 - val_loss: 0.2418 - val_acc: 0.9294 - lr: 0.0010
Epoch 8/1000
400/400 [==============================] - 49s 123ms/step - loss: 0.1708 - acc: 0.9536 - val_loss: 0

In [ ]:
obsession_model.load_weights('model/lstm_obsession_model.h5')
obsession_predictions = obsession_model.predict([padded_docs_test])

41/41 [==============================] - 2s 21ms/step


In [ ]:
print('Obsession Prediction Result')
thresholds=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
for val in thresholds:
    pred=obsession_predictions.copy()

    pred[pred>=val]=1
    pred[pred<val]=0

    precision = precision_score(y_test.values, pred, average='micro')
    recall = recall_score(y_test.values, pred, average='micro')
    f1 = f1_score(y_test.values, pred, average='micro')

    print("Micro-average quality numbers for threshold", val)
    print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

Obsession Prediction Result
Micro-average quality numbers for threshold 0.1
Precision: 0.9190, Recall: 0.9190, F1-measure: 0.9190
Micro-average quality numbers for threshold 0.2
Precision: 0.9335, Recall: 0.9335, F1-measure: 0.9335
Micro-average quality numbers for threshold 0.3
Precision: 0.9343, Recall: 0.9343, F1-measure: 0.9343
Micro-average quality numbers for threshold 0.4
Precision: 0.9351, Recall: 0.9351, F1-measure: 0.9351
Micro-average quality numbers for threshold 0.5
Precision: 0.9328, Recall: 0.9328, F1-measure: 0.9328
Micro-average quality numbers for threshold 0.6
Precision: 0.9251, Recall: 0.9251, F1-measure: 0.9251
Micro-average quality numbers for threshold 0.7
Precision: 0.9167, Recall: 0.9167, F1-measure: 0.9167
Micro-average quality numbers for threshold 0.8
Precision: 0.9022, Recall: 0.9022, F1-measure: 0.9022
Micro-average quality numbers for threshold 0.9
Precision: 0.8808, Recall: 0.8808, F1-measure: 0.8808


In [ ]:
Y_test = y_test.copy()
Y_test['obsession_pred'] = obsession_predictions

In [ ]:
Y_test[:40]

### Prediction using manual label dataset

In [ ]:
manual = pd.read_csv('/content/drive/MyDrive/post classification/processed_data/post-manual-label.csv')

In [ ]:
# Assuming 'post' is the column containing text data
manual['post'] = manual['post'].fillna("")  # Fill missing values with an empty string
manual['post'] = manual['post'].apply(process_text)
manual.head()

,post,prefLabel,obsession_first_checker,compulsion_first_checker,expand1
0,grew obsessive thought aptly intrusive thought...,"['site', 'compulsion', 'obsession', 'rash', 'p...",1.0,1.0,"Aggressive obsession, Contamination obsession,..."
1,thinking much covid part job well right concer...,"['singing', 'washing hands', 'anxiety', 'anxie...",0.0,1.0,"morbid thoughts, Mental image, Homicidal thoug..."
2,fed send day checking excessive checking every...,"['OCD', 'checking', 'Thought']",0.0,1.0,"morbid thoughts, Mental image, Homicidal thoug..."
3,noticed washing hand lately thing triggering n...,"['washing hands', 'hope', 'depression', 'OCD']",1.0,0.0,NaN
4,sound stupid moment want download window live ...,['Thought'],1.0,1.0,"morbid thoughts, Mental image, Homicidal thoug..."


In [ ]:
X_test = manual['post']

In [ ]:
encoded_docs_val =  vect.texts_to_sequences(X_test)
padded_docs_val = pad_sequences(encoded_docs_test, maxlen=MAX_LEN, padding='pre')

In [ ]:
obsession_model = get_model()
obsession_model.load_weights('model/lstm_obsession_model.h5')

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 150, 16)           32000     
                                                                 
 lstm_2 (LSTM)               (None, 150, 16)           2112      
                                                                 
 dropout_1 (Dropout)         (None, 150, 16)           0         
                                                                 
 lstm_3 (LSTM)               (None, 8)                 800       
                                                                 
 dense_2 (Dense)             (None, 4)                 36        
                                                                 
 dense_3 (Dense)             (None, 1)                 5         
                                                                 
Total params: 34953 (136.54 KB)
Trainable params: 3495

In [ ]:
obsession_predictions=obsession_model.predict([padded_docs_val])

41/41 [==============================] - 2s 22ms/step


In [ ]:
obsession_gt = manual['obsession_first_checker']
obsession_gt.head()

0    1.0
1    0.0
2    0.0
3    1.0
4    1.0
Name: obsession_first_checker, dtype: float64

In [ ]:
obsession = pd.DataFrame({
    'gt': np.array(obsession_gt.values[:470], dtype=np.float32),
    'pred': np.array(obsession_predictions.squeeze()[:470], dtype=np.float32)
})
obsession.dropna(axis=0, inplace=True)
report = classification_report(obsession['gt'].tolist(), (obsession['pred'] > 0.7).tolist())
print('--------------------obsession report--------------------')
print(report)

--------------------obsession report--------------------
              precision    recall  f1-score   support

         0.0       0.70      0.33      0.45       296
         1.0       0.39      0.75      0.52       171

    accuracy                           0.48       467
   macro avg       0.55      0.54      0.48       467
weighted avg       0.59      0.48      0.47       467



In [ ]:
obsession = pd.DataFrame({
    'gt': np.array(obsession_gt.values[:470], dtype=np.float32),
    'pred': np.array(obsession_predictions.squeeze()[:470], dtype=np.float32)
})

# Drop NaN values
obsession.dropna(axis=0, inplace=True)

# Apply the threshold to convert probabilities to binary predictions
threshold = 0.7
obsession['pred_binary'] = (obsession['pred'] > threshold).astype(int)

# Print the number of data labeled as 0 and 1, and predicted as 0 and 1
count_gt_0 = (obsession['gt'] == 0).sum()
count_gt_1 = (obsession['gt'] == 1).sum()
count_pred_0 = (obsession['pred_binary'] == 0).sum()
count_pred_1 = (obsession['pred_binary'] == 1).sum()

print('Number of data labeled as 0 (ground truth):', count_gt_0)
print('Number of data labeled as 1 (ground truth):', count_gt_1)
print('Number of data predicted as 0:', count_pred_0)
print('Number of data predicted as 1:', count_pred_1)

# Print the classification report
report = classification_report(obsession['gt'].tolist(), obsession['pred_binary'].tolist())
print('--------------------obsession report--------------------')
print(report)


Number of data labeled as 0 (ground truth): 296
Number of data labeled as 1 (ground truth): 171
Number of data predicted as 0: 139
Number of data predicted as 1: 328
--------------------obsession report--------------------
              precision    recall  f1-score   support

         0.0       0.70      0.33      0.45       296
         1.0       0.39      0.75      0.52       171

    accuracy                           0.48       467
   macro avg       0.55      0.54      0.48       467
weighted avg       0.59      0.48      0.47       467



# Using merged data for training and testing

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/post classification/processed_data/posts-ontology-manual-label.csv')
df

,post,prefLabel,compulsion,obsession,expand1
0,Hi -\nSo I haven't been on here since December...,"['depression', 'weight gain', 'Medication', 'a...",0,1,"Aggressive obsession, Contamination obsession,..."
1,"Hi all, hope you're all having a wonderful ban...","['compulsion', 'anger', 'symptom', 'compassion...",1,1,"Aggressive intrusive thoughts, Contamination i..."
2,"Hi, \nFirst, I hope everyone managed to have s...","['hope', 'happiness', 'guilt', 'fear', 'obsess...",0,1,"Aggressive obsession, Contamination obsession,..."
3,Hello everyone. I could really use your help r...,"['Treatment', 'hope', 'Thought', 'obsession', ...",0,1,"Aggressive obsession, Contamination obsession,..."
4,"Though it comes in many flavors, one of the mo...","['quality', 'Intrusive thoughts', 'fall', 'beh...",1,1,"Aggressive obsession, Contamination obsession,..."
...,...,...,...,...,...
9991,https://itunes.apple.com/gb/podcast/dhamma-tal...,"['interest', 'Thought', 'OCD', 'cognitive proc...",0,0,"morbid thoughts, Mental image, Homicidal thoug..."
9992,Hi all\nThere are lots of helpful topics on th...,['OCD'],0,0,NaN
9993,"aww guys i am sorry , i feel that i am always ...","['OCD', 'sadness', 'weakness', 'anxiety', 'anx...",0,0,"castration anxiety, performance anxiety, koro,..."
9994,My name is Ian I have suffered with OCD for mo...,"['OCD', 'hope', 'fear']",1,0,"terror, panic, panic, moral panic"


In [ ]:
df['post'] = df['post'].fillna('')
df['post'] = df['post'].apply(process_text)
df.head()

,post,prefLabel,compulsion,obsession,expand1
0,hi since december since recovering prozac sinc...,"['depression', 'weight gain', 'Medication', 'a...",0,1,"Aggressive obsession, Contamination obsession,..."
1,hi hope wonderful bank holiday whatever howeve...,"['compulsion', 'anger', 'symptom', 'compassion...",1,1,"Aggressive intrusive thoughts, Contamination i..."
2,hi first hope everyone managed joy happiness c...,"['hope', 'happiness', 'guilt', 'fear', 'obsess...",0,1,"Aggressive obsession, Contamination obsession,..."
3,hello everyone could really use help right ide...,"['Treatment', 'hope', 'Thought', 'obsession', ...",0,1,"Aggressive obsession, Contamination obsession,..."
4,though come many flavor one common ocd theme s...,"['quality', 'Intrusive thoughts', 'fall', 'beh...",1,1,"Aggressive obsession, Contamination obsession,..."


In [ ]:
X, Y = df['post'], df[['obsession']]
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((7996,), (7996, 1), (2000,), (2000, 1))

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
num_words = 3000
vect=Tokenizer(num_words=num_words)
vect.fit_on_texts(X_train)
vocab_size = len(vect.word_index) + 1
print(vocab_size)

22629


In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
encoded_docs_train = vect.texts_to_sequences(X_train)
MAX_LEN = 150
padded_docs_train = pad_sequences(encoded_docs_train, maxlen=MAX_LEN, padding='pre')
padded_docs_train.shape

(7996, 150)

In [ ]:
encoded_docs_test =  vect.texts_to_sequences(X_test)
padded_docs_test = pad_sequences(encoded_docs_test, maxlen=MAX_LEN, padding='pre')

In [ ]:
X_test.shape

(2000,)

In [ ]:
# Index of the document you want to print (change this to the desired document)
document_index = 0

# Retrieve the original text document
original_text = X_train[document_index]

# Retrieve the numerical representation of the document
numerical_representation = padded_docs_train[document_index]

# Print the document and its numerical representation
print("Original Text Document:")
print(original_text)

print("\nNumerical Representation:")
print(numerical_representation)

In [ ]:
from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout

def get_model():
    model = Sequential()
    # Configuring the parameters
    model.add(Embedding(num_words, output_dim=16, input_length=MAX_LEN))
    model.add(LSTM(16, return_sequences=True))
    # Adding a dropout layer
    model.add(Dropout(0.5))
    model.add(LSTM(8))
    # model.add(Dropout(0.5))
    model.add(Dense(4))
    # model.add(Dropout(0.5))
    # Adding a dense output layer with sigmoid activation
    model.add(Dense(1, activation='sigmoid'))
    print(model.summary())
    return model

In [ ]:
obsession_model = get_model()
obsession_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
history = obsession_model.fit(padded_docs_train, y_train, epochs=1000, batch_size=16,validation_split=0.2,
                    callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss',patience=20, min_delta=1e-7),
                              keras.callbacks.ReduceLROnPlateau(factor=0.2, patience=10),
                              keras.callbacks.ModelCheckpoint(filepath='model/lstm_obsession_model.h5',
                                      monitor='val_loss',
                                      save_best_only=True)])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 150, 16)           48000     
                                                                 
 lstm (LSTM)                 (None, 150, 16)           2112      
                                                                 
 dropout (Dropout)           (None, 150, 16)           0         
                                                                 
 lstm_1 (LSTM)               (None, 8)                 800       
                                                                 
 dense (Dense)               (None, 4)                 36        
                                                                 
 dense_1 (Dense)             (None, 1)                 5         
                                                                 
Total params: 50953 (199.04 KB)
Trainable params: 50953 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


400/400 [==============================] - 43s 108ms/step - loss: 0.2121 - acc: 0.9353 - val_loss: 0.2186 - val_acc: 0.9337 - lr: 0.0010
Epoch 3/1000
400/400 [==============================] - 45s 111ms/step - loss: 0.1609 - acc: 0.9543 - val_loss: 0.2393 - val_acc: 0.9300 - lr: 0.0010
Epoch 4/1000
400/400 [==============================] - 46s 114ms/step - loss: 0.1476 - acc: 0.9601 - val_loss: 0.2179 - val_acc: 0.9356 - lr: 0.0010
Epoch 5/1000
400/400 [==============================] - 42s 105ms/step - loss: 0.1239 - acc: 0.9658 - val_loss: 0.2181 - val_acc: 0.9344 - lr: 0.0010
Epoch 6/1000
400/400 [==============================] - 44s 110ms/step - loss: 0.1201 - acc: 0.9679 - val_loss: 0.2650 - val_acc: 0.9150 - lr: 0.0010
Epoch 7/1000
400/400 [==============================] - 42s 106ms/step - loss: 0.1451 - acc: 0.9492 - val_loss: 0.2701 - val_acc: 0.9181 - lr: 0.0010
Epoch 8/1000
400/400 [==============================] - 44s 109ms/step - loss: 0.0971 - acc: 0.9697 - val_loss: 0

In [ ]:
obsession_predictions=obsession_model.predict([padded_docs_test])

63/63 [==============================] - 3s 23ms/step


In [ ]:
# Assuming 'obsession_predictions' contains probability scores
binary_predictions = np.round(obsession_predictions)

In [ ]:
print(type(y_test), type(obsession_predictions))

<class 'pandas.core.frame.DataFrame'> <class 'numpy.ndarray'>


In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Assuming 'y_test' is a Pandas DataFrame
y_test_array = y_test.to_numpy()

# Assuming 'obsession_predictions' contains the predicted labels or probabilities
# Convert predicted probabilities to binary class labels if needed
binary_predictions = np.round(obsession_predictions)

# Evaluate accuracy
accuracy = accuracy_score(y_test_array, binary_predictions)
print(f'Accuracy: {accuracy:.4f}')

# Generate a classification report
class_report = classification_report(y_test_array, binary_predictions)
print('Classification Report:\n', class_report)

# Generate a confusion matrix
conf_matrix = confusion_matrix(y_test_array, binary_predictions)
print('Confusion Matrix:\n', conf_matrix)


Accuracy: 0.9055
Classification Report:
               precision    recall  f1-score   support

           0       0.87      0.95      0.91       996
           1       0.94      0.86      0.90      1004

    accuracy                           0.91      2000
   macro avg       0.91      0.91      0.91      2000
weighted avg       0.91      0.91      0.91      2000

Confusion Matrix:
 [[944  52]
 [137 867]]
